In [9]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [10]:
# Đọc file CSV
file_path = 'product_details.csv'  # Đường dẫn file
data = pd.read_csv(file_path)

# Hiển thị thông tin tổng quan về dữ liệu
print("Thông tin tổng quan về dữ liệu:")
print(data.info())
display(data.head())

Thông tin tổng quan về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1070 non-null   object
 1   store_name     1068 non-null   object
 2   rating         1070 non-null   object
 3   comment_count  1070 non-null   object
 4   price          1070 non-null   object
 5   sold           1070 non-null   object
 6   link           1070 non-null   object
dtypes: object(7)
memory usage: 58.6+ KB
None


,product_name,store_name,rating,comment_count,price,sold,link
0,"[HCM]Thẻ nhớ 64gb, Thẻ nhớ camera YOOSEE Micro...",Không tìm thấy cửa hàng,0,Không có đánh giá,"₫84,800",594 sold,https://www.lazada.vn/products/hcmthe-nho-64gb...
1,"[HCM]Thẻ nhớ 32gb, Thẻ nhớ camera YOOSEE Micro...",Không tìm thấy cửa hàng,0,Không có đánh giá,"₫66,000",3.8K sold,https://www.lazada.vn/products/hcmthe-nho-32gb...
2,Thẻ nhớ TF 16GB 32GB 64GB 128GB 256GB 512GB 1T...,Không tìm thấy cửa hàng,0,Không có đánh giá,"₫49,000",561 sold,https://www.lazada.vn/products/the-nho-tf-16gb...
3,Thẻ Nhớ Kioxia 32GB và Thẻ Nhớ Sandisk (64GB/1...,EZVIZ VN Authorized Store,5.0,750 đánh giá,"₫159,000",2.8K sold,https://www.lazada.vn/products/the-nho-kioxia-...
4,Thẻ Nhớ 32 gb ( Thẻ nhớ dành chuyên cho camera...,Không tìm thấy cửa hàng,0,Không có đánh giá,"₫25,000",96 sold,https://www.lazada.vn/products/the-nho-32-gb-t...


In [11]:
# Bước 1: Kiểm tra và xử lý giá trị trùng lặp
print("\nSố lượng giá trị trùng lặp trước khi xử lý:", data.duplicated().sum())
data = data.drop_duplicates()
print("Số lượng giá trị trùng lặp sau khi xử lý:", data.duplicated().sum())


Số lượng giá trị trùng lặp trước khi xử lý: 34
Số lượng giá trị trùng lặp sau khi xử lý: 0


In [12]:
# Bước 2: Kiểm tra và xử lý giá trị thiếu
missing_values = data.isnull().sum()
print("\nSố lượng giá trị thiếu trong mỗi cột:")
print(missing_values)


Số lượng giá trị thiếu trong mỗi cột:
product_name     0
store_name       2
rating           0
comment_count    0
price            0
sold             0
link             0
dtype: int64


#### Xử lý giá trị thiếu:

In [13]:
# Loại bỏ các dòng có giá trị thiếu
print("\nSố lượng dòng dữ liệu trước khi xử lý store_name:", len(data))
data = data.dropna(subset=['store_name'])
print("Số lượng dòng dữ liệu sau khi xử lý store_name:", len(data))



Số lượng dòng dữ liệu trước khi xử lý store_name: 1036
Số lượng dòng dữ liệu sau khi xử lý store_name: 1034


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1034 entries, 0 to 1069
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1034 non-null   object
 1   store_name     1034 non-null   object
 2   rating         1034 non-null   object
 3   comment_count  1034 non-null   object
 4   price          1034 non-null   object
 5   sold           1034 non-null   object
 6   link           1034 non-null   object
dtypes: object(7)
memory usage: 64.6+ KB


In [15]:
# - Chuyển đổi 'price' và 'sold' sang dạng số
data['price'] = pd.to_numeric(data['price'].str.replace('₫', '').str.replace(',', ''), errors='coerce')
data['sold'] = data['sold'].str.replace(' sold', '').str.replace('K', 'e3')
data['sold'] = pd.to_numeric(data['sold'], errors='coerce')


data = data.dropna(subset=['price', 'sold']) 

In [16]:
# Kiểm tra các giá trị không hợp lệ trong 'price' và 'sold'
print("Giá trị không hợp lệ trong 'price':")
print(data[~data['price'].str.replace('₫', '').str.replace(',', '').str.isnumeric()])  # Kiểm tra giá trị không hợp lệ

print("Giá trị không hợp lệ trong 'sold':")
print(data[~data['sold'].str.replace(' sold', '').str.replace('K', 'e3').str.isnumeric()])  # Kiểm tra giá trị không hợp lệ

# Xử lý các giá trị không hợp lệ (nếu cần)
data['price'] = data['price'].str.replace('₫', '').str.replace(',', '').replace('', np.nan)  # Thay thế giá trị rỗng bằng NaN
data['sold'] = data['sold'].str.replace(' sold', '').str.replace('K', 'e3').replace('', np.nan)  # Thay thế giá trị rỗng bằng NaN

# Chuyển đổi sang dạng số
data['price'] = pd.to_numeric(data['price'], errors='coerce')
data['sold'] = pd.to_numeric(data['sold'], errors='coerce')

# Loại bỏ các dòng có giá trị null trong 'price' hoặc 'sold'
data = data.dropna(subset=['price', 'sold'])

Giá trị không hợp lệ trong 'price':


AttributeError: Can only use .str accessor with string values!

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1033 entries, 0 to 1069
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_name   1033 non-null   object 
 1   store_name     1033 non-null   object 
 2   rating         1033 non-null   object 
 3   comment_count  1033 non-null   object 
 4   price          1033 non-null   float64
 5   sold           1033 non-null   float64
 6   link           1033 non-null   object 
dtypes: float64(2), object(5)
memory usage: 64.6+ KB


In [18]:
# Xử lý comment_count
# - Thay thế giá trị "không có bình luận" bằng NaN
data['comment_count'] = data['comment_count'].replace('Không có đánh giá', np.nan)

# - Trích xuất số lượng bình luận từ chuỗi
data['comment_count'] = data['comment_count'].str.extract(r'(\d+)').astype(float)

data = data.dropna(subset=['comment_count'])

data.head()

,product_name,store_name,rating,comment_count,price,sold,link
3,Thẻ Nhớ Kioxia 32GB và Thẻ Nhớ Sandisk (64GB/1...,EZVIZ VN Authorized Store,5.0,750.0,159000.0,2800.0,https://www.lazada.vn/products/the-nho-kioxia-...
9,♥COD + Thẻ Nhớ Micro SD TOSHIBA Chính Hãng Chí...,sbiqdn store,4.6,1016.0,32000.0,4700.0,https://www.lazada.vn/products/cod-the-nho-mic...
16,💞【Readystock】 + Giao hàng miễn phí + cá tuyết💞...,AUGUST-MALL ZERO,4.5,95.0,69000.0,281.0,https://www.lazada.vn/products/readystock-giao...
21,Sony Thẻ nhớ Micro SD 32GB/64GB/128GB/256GB/51...,Electronic Collection Shop,4.7,36.0,75000.0,48.0,https://www.lazada.vn/products/sony-the-nho-mi...
25,Thẻ nhớ 2G Thẻ Nhớ Micro SD Dung Lượng 2G Giao...,KEN warehouse,5.0,3.0,22000.0,41.0,https://www.lazada.vn/products/the-nho-2g-the-...


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 549 entries, 3 to 1068
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_name   549 non-null    object 
 1   store_name     549 non-null    object 
 2   rating         549 non-null    object 
 3   comment_count  549 non-null    float64
 4   price          549 non-null    float64
 5   sold           549 non-null    float64
 6   link           549 non-null    object 
dtypes: float64(3), object(4)
memory usage: 34.3+ KB


In [20]:
# Bước 3: Chuẩn hóa dữ liệu
# Chỉ chuẩn hóa các cột số
numeric_cols = ['price', 'sold', 'estimated_revenue']
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[numeric_cols])

# Chuyển dữ liệu đã chuẩn hóa thành DataFrame
scaled_df = pd.DataFrame(data_scaled, columns=[f"{col}_scaled" for col in numeric_cols])

# Kết hợp dữ liệu gốc và dữ liệu đã chuẩn hóa
final_data = pd.concat([data.reset_index(drop=True), scaled_df], axis=1)



# Bước 3: Lưu dữ liệu vào file CSV duy nhất
output_path = 'preprocessing.csv'
final_data.to_csv(output_path, index=False)
print(f"\nDữ liệu cuối cùng đã được lưu tại: {output_path}")



KeyError: "['estimated_revenue'] not in index"